## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
stat = pd.read_csv('/datasets/data.csv')
stat.info() # смотрим общую информацию начального набора данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
stat.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
stat['debt'].value_counts() # смотрим уникальные значения в ключевом столбце, по которому будем вычислять нашу метрику

0    19784
1     1741
Name: debt, dtype: int64

### Вывод

Всего в начальном наборе данных stat 21525 строк. Все столбцы stat заполнены непустыми данными, кроме двух: 'days_employed' и 'total_income'. В этих двух столбцах отсутствует одинаковое(!) кол-во данных (около 10% данных в каждом столбце). 

Также есть явно некорректные значения в столбце 'days_employed' - отрицательные значения и слишком большие, если судить по названию столбца - days_employed

Для ответа на поставленные вопросы будем использовать метрику на основе данных столбца 'debt'. Нашей метрикой будет отношение количества единиц (невозвратов) к общему количеству значений. Это значит, что на агрегированых данных метрику можно считать sum()/count() 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# изучим только те строки stat, в которых нет данных в столбце 'total_income'
stat[stat['total_income'].isnull()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Мы видим, что пропуски 'days_employed' и 'total_income' в одних и тех же строках. Рабочая гипотеза - пропуски это на самом деле нули, т.е. это данные людей, которые не работают и не имеют постоянного дохода. Посмотрим, есть ли в столбцах 'days_employed' и 'total_income' нули

In [6]:
stat_notnull = stat[stat['total_income'].notnull()] # выделяемм строки без пропущенных значений
print(stat_notnull[stat_notnull['total_income'] == 0]['total_income'].count()) # посчитаем нули в 'total_income' 
print(stat_notnull[stat_notnull['days_employed'] == 0]['days_employed'].count()) # посчитаем нули в 'days_employed' 

0
0


Вроде бы гипотеза подтверждается, и можно заменить пропуски на 0. Но все же проверим гипотезу еще одним способом. Из общих соображений мы понимаем, что люди без заработка должны чаще не возращать кредиты. Поэтому метрика для всех данных и метрика для данных с пропусками должны заметно отличаться

In [7]:
# считаем метрику на всех данных :
debts_total = stat['debt'].count() # считаем все кредиты
debts_bad = stat['debt'].sum() # считаем количество невозвратов
metric_all = debts_bad / debts_total 
print("Метрика для всех данных        ", metric_all)
# аналогично считаем метрику на данных с пропусками:
stat_none = stat[stat['total_income'].isnull()] # выберем строки с пропусками
debts_total = stat_none['debt'].count() 
debts_bad = stat_none['debt'].sum() 
metric_none = debts_bad / debts_total 
print("Метрика для данных c пропусками", metric_none)
print("Отличие в метриках             ", (metric_all - metric_none) / metric_all)

Метрика для всех данных         0.08088269454123112
Метрика для данных c пропусками 0.078196872125115
Отличие в метриках              0.03320639144566309


Наша метрика для всех данных и метрика для данных с пропусками отличаются на 3.3%
Отличие есть, но небольшое. На этом этапе анализа непонятно, 
можно ли удалить строки с пропусками или заменить пропуски на 0. 

Можно пропуски заменить на средние значения (арифметические или медианные), 
однако на этом этапе непонятно, коррктно ли заменять пропуски на средние по всем данным.

Присвоим пропускам 0. Если в дальнейшем поймем, что это не согласуется с другими данными, 
сможем 0 поменять на что-то более подходящее - ведь в начальном наборе вообще не было нулей в этих столбцах.

In [8]:
# присваиваем 0 пропускам в столбцах total_income и days_employed
stat['total_income'] = stat['total_income'].fillna(value=0)
stat['days_employed'] = stat['days_employed'].fillna(value=0)
stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
# на этапе знакомства с данными нас смутило большое кол-во отрицательных и больших значений в столбце days_employed           
# посмотрим насколько много слишком больших значений или отрицательных значений
def days_employed_to_category(val):
    days_too_many = 365 * 50
    if val < -days_too_many:
        return 'negative too big'
    elif val < 0:
        return 'negative'    
    elif val == 0:
        return 'processed'
    elif val < days_too_many:
        return 'normal'
    else:
        return 'too big'
stat['days_employed_category'] = stat['days_employed'].apply(days_employed_to_category)
stat['days_employed_category'].value_counts()

negative            15905
too big              3445
processed            2174
negative too big        1
Name: days_employed_category, dtype: int64

In [10]:
# видно, что в этом столбце либо отрицательные значения (причем по модулю не слишком большие), 
# а если положительные, тогда они чрезмерно большие
stat.groupby('days_employed_category')['days_employed'].median()

days_employed_category
negative             -1629.997862
negative too big    -18388.949901
processed                0.000000
too big             365213.306266
Name: days_employed, dtype: float64

Данные в этом столбце не будут использовать нами для анализа, однако зафиксируем явные ошибки в данных:

- есть отрицательные значения, но, вероятно, если их взять по модулю, получим корректные значения
- все положительные значения слишком большие, чтобы представлять из себя количество стажа в днях

In [11]:

# Теперь посмотрим на распределение по уникальным значениям в тех столбцах, которые являются ключевыми 
# для ответов на поставленные вопросы - нас интересуют категориальные столбцы 'family_status', 'children' и 'purpose'

# распределение по уникальным значениям 'family_status'
stat['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [12]:
# В столбце 'family_status' все данные выглядят корректными, неявных пропусков и заведомо недостоверных данных нет
# распределение по уникальным значениям 'children'
stat['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [13]:
# видим два странных значения: 20 и -1, посмотрим на данные с этими значениями 
stat[stat['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_category
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья,negative
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости,negative
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования,negative
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью,negative
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль,negative
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование,negative
21390,20,0.000000,53,среднее,1,женат / замужем,0,M,компаньон,0,0.000000,покупка жилой недвижимости,processed
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью,negative


In [14]:
stat[stat['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_category
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование,negative
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля,negative
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование,negative
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование,too big
941,-1,0.000000,57,Среднее,1,женат / замужем,0,F,пенсионер,0,0.000000,на покупку своего автомобиля,processed
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование,negative
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья,negative
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья,negative
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы,negative
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью,negative


In [15]:
# Строк с артефактами немного (около 0,5%), явного правила замены этих данных не просматривается, 
# поэтому мы их удалим из дальнейшего анализа.
stat = stat[stat['children'] != - 1]
stat = stat[stat['children'] != 20]
stat['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [16]:
stat['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

В столбце 'purpose' каких-либо явно ошибочных значений не видно, но есть неочевидные дубликаты, которые мы обработаем на следующих этапах

### Вывод

- Анализ данных показал, что пропуски в столбцах "total_income" и "days_employed" имеют общую причину - пропуски в обоих столбцах в одних и тех же строках. Строк с такими пропусками заметное число - чуть более 10%. Возможно, пропуски в этих столбцах это некорректный ввод 0. Но на этом этапе анализа до конца непонятно, какая замена пропусков будет адекватной. Сейчас пропускам в этих столбцах присвоили нули. В дальнейшем наше решение можно будет легко изменить, так как в начальном наборе вообще не было нулей в этих столбцах. 

- Обнаружены артефакты в столбце "children" - значения 20 и -1, суммарно их немного: всего 123 строки, или ~0.5%, какая замена будет адекватной непонятно, поэтому просто их удалили. 

- Данные в столбце 'family_status' корректны.

- Данные в столбце 'purpose' с точки зрения пропусков корректны. Есть явные дубликаты, займемся ими на следующем этапе обработки

- Вопрос с пропусками в столбцах "total_income" и "days_employed" необходимо обсудить с коллегами, ответственными за сбор данных.

- Обязательно надо обсудить некорректность данных в столбце "days_employed" - там есть либо пропуски, либо отрицательные значения, либо слишком большие положительные значения 

### Замена типа данных

In [17]:
# в столбцах "total_income" и "days_employed" данные преобразуем вещественный тип в целый
for column in ['total_income', 'days_employed']:
    stat[column] = stat[column].astype('int')
stat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   children                21402 non-null  int64 
 1   days_employed           21402 non-null  int64 
 2   dob_years               21402 non-null  int64 
 3   education               21402 non-null  object
 4   education_id            21402 non-null  int64 
 5   family_status           21402 non-null  object
 6   family_status_id        21402 non-null  int64 
 7   gender                  21402 non-null  object
 8   income_type             21402 non-null  object
 9   debt                    21402 non-null  int64 
 10  total_income            21402 non-null  int64 
 11  purpose                 21402 non-null  object
 12  days_employed_category  21402 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


### Вывод

В столбцах "total_income" и "days_employed" данные имеют вещественный тип. Очевидно, нам не нужна такая точность в этих столбцах. К тому же для вывода о влиянии дохода на возврат кредита нам необходимо будет категоризировать данные, разбив по группам доходов. Это быстрее делать с целыми числами. Преобразовали данные в этих столбцах в целые числа методом astype

### Обработка дубликатов

Мы исследовали все важные для нас столбцы с категориальными данными на стадии поиска пропусков, используя метод count_values(). Сейчас дополнительно проверим данные только на полное совпадение, причем исключим из проверки те строки, в которых пропуски заполнили вручную (помним, что значения в пропусках могли быть разными).  

In [18]:
stat[stat['total_income'] != 0].duplicated().sum()

0

In [19]:
# посмотрим для сведения полные дубликаты для всех данных, 
# включая строки, в которых пропуски заполнили вручную 
stat.duplicated().sum()

54

In [20]:
# посмотрим, что с данными в столбце  'debt' для этих 54 строк
stat[stat.duplicated()]['debt'].value_counts()

0    54
Name: debt, dtype: int64

В этих строках вообще нет невозвратов кредитов, для средних значений должно было быть 4-5 невозврата, но такое расхождение со средним некритично. Если бы здесь были только невозвраты, тогда беспокойства было бы больше, сейчас же просто оставим эти строки, учитывая, что дубликаты возникли только в строках с вручную заполненными пропусками в столбцах 'income' и 'days_employed'

### Вывод

Полных дубликатов, если исключить строки с нулевыми значения в столбцах 'total_income' и 'days_employed', нет

Мы исследовали на этапе поиска пропусков столбцы с помощью метода count_values(). Дубликатов, связанных c разным написанием одинаковых фраз/слов, в столбцах 'children', 'family_status', нет. Мы видим неявные дубликаты в столбце 'purpose', но обработаем их в дальнейшем после лемматизации

### Лемматизация

Нам необходимо удалить неявные дубликаты и категоризировать столбец 'purpose'. Уникальных значений в нем чуть более полусотни, но каждое уникальное значение представляет из себя фразу из нескольких слов. Можно, конечно, сделать все это вручную, но даже на пятидесяти фразах это трудоемко. К тому же мы предполагаем, что категории из этой колонки могут формироваться различными способами, поэтому необходима автоматизация процесса. 

Для возможности фильтрации по ключевым словам проведем лемматизацию

In [21]:
# лемматизация процесс ресурсоемкий, поэтому будем проводить лемматизацию 
# только уникальных значений столбца 'purpose' 
dict_purpose = stat[['purpose']] # выбираем столбец
dict_purpose = dict_purpose.drop_duplicates().reset_index(drop=True) # сокращаем повторения 

from pymystem3 import Mystem
m = Mystem()

# для лемматизации столбца 'purpose' создадим функцию:
def lemma_column(str):
    lemmas = m.lemmatize(str)
    return lemmas

# лемматизированный 'purpose' сохраним в отдельном столбце
dict_purpose['purpose_lemma'] = dict_purpose['purpose'].apply(lemma_column)
dict_purpose

,purpose,purpose_lemma
0,покупка жилья,"[покупка, , жилье, \n]"
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,дополнительное образование,"[дополнительный, , образование, \n]"
3,сыграть свадьбу,"[сыграть, , свадьба, \n]"
4,операции с жильем,"[операция, , с, , жилье, \n]"
5,образование,"[образование, \n]"
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"
8,покупка недвижимости,"[покупка, , недвижимость, \n]"
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]"


In [22]:
# для изучения выборок по ключевым словам создадим функцию is_containing()
keyword_FOR_SEARCH_in_lemmas = '' # мы будем использовать эту переменную как константу в функции is_containing()

def is_containing(list_lemmas):
    if keyword_FOR_SEARCH_in_lemmas in list_lemmas:
        return True
    return False

# изучим более внимательно различные значения 'purpose' по ключевым словам:
for keyword_check in ['недвижимость', 'жилье', 'автомобиль', 'свадьба', 'образование', 'коммерческий', 'ремонт', 'сдача']:
    keyword_FOR_SEARCH_in_lemmas = keyword_check
    print('Ключевое слово:', keyword_check)
    print(dict_purpose[dict_purpose['purpose_lemma'].apply(is_containing)]['purpose'])
    print('________')

Ключевое слово: недвижимость
8                       покупка недвижимости
9          покупка коммерческой недвижимости
10                покупка жилой недвижимости
11    строительство собственной недвижимости
12                              недвижимость
13                строительство недвижимости
16     операции с коммерческой недвижимостью
17          строительство жилой недвижимости
19           операции со своей недвижимостью
28                  операции с недвижимостью
Name: purpose, dtype: object
________
Ключевое слово: жилье
0               покупка жилья
4           операции с жильем
7     покупка жилья для семьи
18                      жилье
27       покупка своего жилья
34    покупка жилья для сдачи
36               ремонт жилью
Name: purpose, dtype: object
________
Ключевое слово: автомобиль
1                приобретение автомобиля
14    на покупку подержанного автомобиля
15          на покупку своего автомобиля
20                            автомобили
22      сделка с подер

### Вывод

Мы видим, что в столбце 'purpose' есть синонимы: например, "покупка жилья для семьи" синоним "покупка жилой недвижимости". Однако есть вроде бы и синонимы, но мы, возможно, захотим отнести их к разным категориям. Например, значения: 'недвижимость', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилья для сдачи' - можно либо объединить в одну категорию "недвижимость", либо разбить на две категории "жилая недвижимость", "коммерческая недвижимость".

Мы не будем сокращать количество записей в словаре целей, для категоризации создадим новый столбец. В дальнейшем, можем возвращаться в эту точку для перекатегоризации целей получения кредита.

### Категоризация данных

In [23]:
# Категоризацию по столбцу 'purpose', возможно, придется менять несколько раз, 
# чтобы получить аналитку по разным наборам категорий.
# Создадим категоризирующую функцию purpose_to_category() так, чтобы можно было легко изменить категоризацию.
# 
# Логика следующая: 
# выбор категории осуществляется максимум по двум ключевым словам, key_1 и key_2, key_2 является необязательным
# в словаре purpose_glossary с индексом key_1 сохраняем подсловари с индексом key_2
# если key_1 находится в леммах столбца 'purpose', берем подсловарь purpose_glossary[key_1]
# если в подсловаре всего одно значение (для данного key_1 нет key_2), берем значение с индексом 0
# если в подсловаре не одно значение (есть key_2), тогда  ищем в леммах key_2 
# если находим, тогда берем значение категории из purpose_glossary[key_1][key_2]
#
# поиск по key_2 включает поиск по 0, это и лишние траты ресурсов 
# и, главное, подводный камень в логике для будущего использования (если вдруг в леммах будет 0),
# поэтому исключаем явно этот поиск из цикла for..in при переборе ключей в подсловаре

purpose_glossary = {   
    'недвижимость': {
                                  0: 'недвижимость жилая',
                     'коммерческий': 'недвижимость коммерческая',
                     },
           'жилье': {
                                 0 : 'недвижимость жилая',
                           'ремонт': 'недвижимость жилая, ремонт',
                            'сдача': 'недвижимость коммерческая',
                     },
      'автомобиль': {0 : 'автомобиль'},
         'свадьба': {0 : 'свадьба'},
     'образование': {
                                 0 : 'образование вообще',
                   'дополнительный': 'образование дополнительное',             
                     },
    }
def purpose_to_category(list_lemmas): # создаем функцию для категоризации по ключевым словам в лемматизированном 'purpose'
    for key in purpose_glossary.keys():
        if key in list_lemmas:
            if len(purpose_glossary[key]) == 1:
                return purpose_glossary[key][0]
            else:
                for key_internal in purpose_glossary[key].keys():
                    if key_internal != 0: # исключаем проверку в леммах для 0, так как 0 это дефолтное значение словаря
                        if key_internal in list_lemmas:
                            return purpose_glossary[key][key_internal]
                return purpose_glossary[key][0]
    return 'остальное'

        

dict_purpose['purpose_category'] = dict_purpose['purpose_lemma'].apply(purpose_to_category)
dict_purpose

,purpose,purpose_lemma,purpose_category
0,покупка жилья,"[покупка, , жилье, \n]",недвижимость жилая
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,дополнительное образование,"[дополнительный, , образование, \n]",образование дополнительное
3,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
4,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость жилая
5,образование,"[образование, \n]",образование вообще
6,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба
7,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость жилая
8,покупка недвижимости,"[покупка, , недвижимость, \n]",недвижимость жилая
9,покупка коммерческой недвижимости,"[покупка, , коммерческий, , недвижимость, \n]",недвижимость коммерческая


In [24]:
dict_purpose = dict_purpose[['purpose', 'purpose_category']]
stat = stat.merge(dict_purpose, on='purpose', how='left') # присоединяем столбец с категориями 'purpose' к начальному набору данных
stat.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,negative,недвижимость жилая
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,negative,автомобиль
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,negative,недвижимость жилая
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,negative,образование дополнительное
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,too big,свадьба


In [25]:
# Для категоризирии данных столбца 'total_income' создадим функцию income_to_category().
# Ясно, что категоризировать надо по диапазонам доходов (к примеру, от 0 до 50 тыс, от 50 тыс до 100 тыс и т.д.)
# Поскольку границы и даже кол-во диапазонов придется менять, функцию сделаем незавимой от кол-ва диапазонов и границ :
income_category_glossary = [] # границы диапазонов задаем в тыс. 
def income_to_category(value):
    number_of_spans = len(income_category_glossary) + 1 # диапазонов с учетом краевого эффекта больше на единицу, чем границ
    for span_index in range(number_of_spans): #  цикл на 1 длиннее чем максимальный индекс income_category_glossary - для обработки всех диапазонов
        if span_index >= 9:
            level_string = str(span_index + 1) 
        else:
            level_string = '0' + str(span_index + 1) # строковая переменная level_string нужна для нумерации будущих категорий - в будущем мы хотим сортировать категории по убыванию или возрастанию
        is_first = (span_index == 0)
        if not is_first:
            from_bound = income_category_glossary[span_index - 1]
            from_bound_str = str(from_bound)
        is_last = (span_index == (number_of_spans - 1)) 
        if not is_last:
            up_bound_1000 = income_category_glossary[span_index] * 1000 
            up_bound_str = str(income_category_glossary[span_index])  # для последнего диапазона нет верхней границы, поэтому проверка
        if not is_last:
            if value <= up_bound_1000: # для последнего диапазона up_bound_1000 не определен, но для последнего диапазона и не надо ничего проверять
                if is_first: # возврат для первого диапазона "до первой границы"
                    return level_string + ' - доход менее или равен ' + up_bound_str + ' тыс.' 
                else: # возврат для внутренних диапазонов "от... до"
                    if from_bound * 1000 > up_bound_1000: # проверяем, что границы в списке указаны по возрастанию
                        return 'not_valid_category'
                    else:
                        return level_string + ' - доход от ' + from_bound_str + ' до ' + up_bound_str + ' тыс.'
        else: # возврат для последнего диапазона "более чем"
            return level_string + ' - доход более ' + from_bound_str + ' тыс.'
        

income_category_glossary = [0, 20, 60, 80, 100, 120, 140, 160, 190, 240, 240, 300] # 0 и менее - это данные с пропусками
stat['income_category'] = stat['total_income'].apply(income_to_category)
stat['income_category'].value_counts() # следим чтобы в каждом диапазоне было достаточно данных для получения метрики

10 - доход от 190 до 240 тыс.        2607
09 - доход от 160 до 190 тыс.        2456
06 - доход от 100 до 120 тыс.        2369
07 - доход от 120 до 140 тыс.        2240
05 - доход от 80 до 100 тыс.         2179
01 - доход менее или равен 0 тыс.    2162
08 - доход от 140 до 160 тыс.        2012
12 - доход от 240 до 300 тыс.        1639
13 - доход более 300 тыс.            1473
04 - доход от 60 до 80 тыс.          1464
03 - доход от 20 до 60 тыс.           801
Name: income_category, dtype: int64

In [26]:
# Категорий в income_category достаточно много и вряд ли имеет смысл так сильно разбивать данные.
# Однако мы специально сделали такую частую разбивку, главное чтобы данных в каждой категории было достаточно для статистики
# Далее уже на этапе анализа мы увидим распределение нашей метрики по небольшим интервалам и сможем корректно 
# определить границы больших диапазонов. 
# Зарезервируем дополнительный столбец  'income_category_big', который будет содержать категории для окончательного вывода 
income_category_glossary = [0, 80, 140, 200]
stat['income_category_big'] = stat['total_income'].apply(income_to_category)
stat['income_category_big'].value_counts()

03 - доход от 80 до 140 тыс.         6788
04 - доход от 140 до 200 тыс.        5149
05 - доход более 200 тыс.            5038
02 - доход от 0 до 80 тыс.           2265
01 - доход менее или равен 0 тыс.    2162
Name: income_category_big, dtype: int64

In [27]:
stat.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_category,purpose_category,income_category,income_category_big
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,negative,недвижимость жилая,12 - доход от 240 до 300 тыс.,05 - доход более 200 тыс.
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,negative,автомобиль,06 - доход от 100 до 120 тыс.,03 - доход от 80 до 140 тыс.
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,negative,недвижимость жилая,08 - доход от 140 до 160 тыс.,04 - доход от 140 до 200 тыс.
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,negative,образование дополнительное,12 - доход от 240 до 300 тыс.,05 - доход более 200 тыс.
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,too big,свадьба,08 - доход от 140 до 160 тыс.,04 - доход от 140 до 200 тыс.


In [28]:
# Категоризируем столбец 'children', поскольку мы видели на этапе знакомства с данными, что
# во-первых, есть ошибочные значения -1 и 20,
# во-вторых, строк с 3, 4 и 5 детьми немного и для статистики недостаточно, 
# и мы ограничимся статистикой для категории "3 и более"
def kids_category(number):
    if number < 0:
        return 'unknown'
    elif number == 0:
        return 'без детей'
    elif number == 1:
        return '1 ребенок'
    elif number == 2:
        return '2 детей'
    elif number < 6:
        return '3-5 детей'
    else:
        return 'unknown'
    
stat['children_category'] = stat['children'].apply(kids_category)
stat['children_category'].value_counts()

без детей    14149
1 ребенок     4818
2 детей       2055
3-5 детей      380
Name: children_category, dtype: int64

In [29]:
# Мы категоризировали все данные и готовы проводить анализ. 
# Анализ предполагает создание различных группировок, что ресурсоемко.
# Поэтому оставим для анализа только необходимые нам столбцы и 
# преобразуем тип данных в  столбцах object в category :
column_list_for_analysis = ['children_category', 'family_status', 'income_category', 'income_category_big', 'purpose_category', 'debt']
stat_analysis = stat[column_list_for_analysis]
stat_analysis.head(5)

,children_category,family_status,income_category,income_category_big,purpose_category,debt
0,1 ребенок,женат / замужем,12 - доход от 240 до 300 тыс.,05 - доход более 200 тыс.,недвижимость жилая,0
1,1 ребенок,женат / замужем,06 - доход от 100 до 120 тыс.,03 - доход от 80 до 140 тыс.,автомобиль,0
2,без детей,женат / замужем,08 - доход от 140 до 160 тыс.,04 - доход от 140 до 200 тыс.,недвижимость жилая,0
3,3-5 детей,женат / замужем,12 - доход от 240 до 300 тыс.,05 - доход более 200 тыс.,образование дополнительное,0
4,без детей,гражданский брак,08 - доход от 140 до 160 тыс.,04 - доход от 140 до 200 тыс.,свадьба,0


In [30]:
stat_analysis.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21401
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   children_category    21402 non-null  object
 1   family_status        21402 non-null  object
 2   income_category      21402 non-null  object
 3   income_category_big  21402 non-null  object
 4   purpose_category     21402 non-null  object
 5   debt                 21402 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 13.3 MB


In [31]:
stat_analysis = stat_analysis.copy() # создаем копию данных 

for column_index in range(len(column_list_for_analysis) - 1): # 'debt' не преобразовываем в 'category'
    stat_analysis[column_list_for_analysis[column_index]] = stat_analysis[column_list_for_analysis[column_index]].astype('category')
stat_analysis.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21401
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   children_category    21402 non-null  category
 1   family_status        21402 non-null  category
 2   income_category      21402 non-null  category
 3   income_category_big  21402 non-null  category
 4   purpose_category     21402 non-null  category
 5   debt                 21402 non-null  int64   
dtypes: category(5), int64(1)
memory usage: 445.0 KB


In [32]:
# вывод информации по использованию памяти для целей данной работы лишний, 
# я его сделал для количественой оценки полезности преобразования из object в category - 
# уменьшили объем памяти в 30 раз

# создадим переменную общее кол-во строк с данными для контроля учета всех строк в анализе данных 
number_data_rows = stat_analysis['debt'].count()

### Вывод

Мы полностью подготовили данные для проведения анализа и ответов на поставленные вопросы. Созданные для категоризации функции позволяют достаточно гибко изменять список категорий для каждого из столбцов 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
column_for_analysis = 'children_category'
pivot_t = stat_analysis.groupby(column_for_analysis).agg({'debt': ['sum', 'count']})
pivot_t.columns = ['bad', 'total']

# контроль полноты разбиения на категории :
unprocessed = number_data_rows - pivot_t['total'].sum()
if unprocessed == 0:
    print('Обработан весь набор данных')
else:
    print('Количество необработанных строк:', unprocessed)

print('Метрика - процент невозвращенных кредитов') # выводим метрику - процент невозврата в %
pivot_t['metric'] = pivot_t['bad'] / pivot_t['total'] * 100
pivot_t['metric'] = pivot_t['metric'].round(1) 
pivot_t.sort_values(by='metric')

Обработан весь набор данных
Метрика - процент невозвращенных кредитов


,bad,total,metric
children_category,,,
без детей,1063,14149,7.5
3-5 детей,31,380,8.2
1 ребенок,444,4818,9.2
2 детей,194,2055,9.4


### Вывод

- Самые надежные заемщики - клиенты без детей. Процент невозвратов для них - 7,5%, и это заметно ниже других категорий. 

- Самые ненадежные заемщики - клиенты с одним или двумя детьми. Процент невозвратов кредитов для клиентов с одним или двумя детьми практически одинаков.

- Интересный факт, требующий дополнительного подтверждения на более значительной выборке - клиенты с детьми от 3 и более  возвращают кредиты лучше, чем клиенты с 1-2 детьми. Однако в наших данных только 380 записей о таких клиентах и отличие в показателе невозвратов небольшое - примерно на 10%. Так что необходим дополнительный анализ более значительной выборки

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
column_for_analysis = 'family_status'
pivot_t = stat_analysis.groupby(column_for_analysis).agg({'debt': ['sum', 'count']})
pivot_t.columns = ['bad', 'total']

# контроль полноты разбиения на категории :
unprocessed = number_data_rows - pivot_t['total'].sum()
if unprocessed == 0:
    print('Обработан весь набор данных')
else:
    print('Количество необработанных строк:', unprocessed)

print('Метрика - процент невозвращенных кредитов') # выводим метрику - процент невозврата в %
pivot_t['metric'] = pivot_t['bad'] / pivot_t['total'] * 100
pivot_t['metric'] = pivot_t['metric'].round(1) 
pivot_t.sort_values(by='metric')

Обработан весь набор данных
Метрика - процент невозвращенных кредитов


,bad,total,metric
family_status,,,
вдовец / вдова,63,952,6.6
в разводе,84,1189,7.1
женат / замужем,927,12302,7.5
гражданский брак,385,4160,9.3
Не женат / не замужем,273,2799,9.8


### Вывод

- Самая ненадежная категория заемщиков - не женатые/ не замужние. У них процент невозвратов 9.8%.

- Чуть меньше, однако заметно больше других категорий, процент невозвратов у клиентов, состоящих в гражданском браке - 9.3%. 

- У клиентов, состоящих в официальном браке, процент невозратов 7.5, и это примерно на 20% отличается от показателя невозвратов в гражданских браках.

- У вдовцов и клиентов в разводе проценты невозвратов еще ниже (6.6% и 7.1%), но они не столь заметно отличаются от показателя невозвратов семейных пар (официальных) -  отличие на 6-12%. Это не столь заметная разница на небольшом наборе данных для этих категорий. Тем не менее их показатели заметно лучше не женатых или состоящих в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
column_for_analysis = 'income_category'
pivot_t = stat_analysis.groupby(column_for_analysis).agg({'debt': ['sum', 'count']})
pivot_t.columns = ['bad', 'total']

# контроль полноты разбиения на категории :
unprocessed = number_data_rows - pivot_t['total'].sum()
if unprocessed == 0:
    print('Обработан весь набор данных')
else:
    print('Количество необработанных строк:', unprocessed)

print('Метрика - процент невозвращенных кредитов') # выводим метрику - процент невозврата в %
pivot_t['metric'] = pivot_t['bad'] / pivot_t['total'] * 100
pivot_t['metric'] = pivot_t['metric'].round(1) 
pivot_t

Обработан весь набор данных
Метрика - процент невозвращенных кредитов


,bad,total,metric
income_category,,,
01 - доход менее или равен 0 тыс.,170,2162,7.9
03 - доход от 20 до 60 тыс.,49,801,6.1
04 - доход от 60 до 80 тыс.,125,1464,8.5
05 - доход от 80 до 100 тыс.,180,2179,8.3
06 - доход от 100 до 120 тыс.,196,2369,8.3
07 - доход от 120 до 140 тыс.,193,2240,8.6
08 - доход от 140 до 160 тыс.,185,2012,9.2
09 - доход от 160 до 190 тыс.,218,2456,8.9
10 - доход от 190 до 240 тыс.,199,2607,7.6


In [36]:
# из предыдущей сводной таблицы видны подходящие границы категорий для столбца 'income_category_big'
column_for_analysis = 'income_category_big'
pivot_t = stat_analysis.groupby(column_for_analysis).agg({'debt': ['sum', 'count']})
pivot_t.columns = ['bad', 'total']

# контроль полноты разбиения на категории :
unprocessed = number_data_rows - pivot_t['total'].sum()
if unprocessed == 0:
    print('Обработан весь набор данных')
else:
    print('Количество необработанных строк:', unprocessed)

print('Метрика - процент невозвращенных кредитов') # выводим метрику - процент невозврата в %
pivot_t['metric'] = pivot_t['bad'] / pivot_t['total'] * 100
pivot_t['metric'] = pivot_t['metric'].round(1) 
pivot_t

Обработан весь набор данных
Метрика - процент невозвращенных кредитов


,bad,total,metric
income_category_big,,,
01 - доход менее или равен 0 тыс.,170,2162,7.9
02 - доход от 0 до 80 тыс.,174,2265,7.7
03 - доход от 80 до 140 тыс.,569,6788,8.4
04 - доход от 140 до 200 тыс.,463,5149,9.0
05 - доход более 200 тыс.,356,5038,7.1


### Вывод

- Самая надежная категория заемщиков - клиенты с доходами выше 200 тыс. 

- Самая ненадежная - клиенты с доходом от 140тыс. до 200тыс. Разница в их показателях невозврата 20%.

- Далее, при снижении доходов ниже 140 тыс., процент невозвратов снижается, клиенты с доходами менее 80 тыс. возвращают кредиты лучше, чем клиенты с доходами от 80 тыс. до 140 тыс., а те, в свою очередь, лучше чем клиенты от 140 тыс. до 200 тыс.

Исходя из наших данных, распределение по доходам представляет из себя колоколообразную кривую с максимум показателя невозвратов у клиентов с доходами от 140 тыс. до 200 тыс. 

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
column_for_analysis = 'purpose_category'
pivot_t = stat_analysis.groupby(column_for_analysis).agg({'debt': ['sum', 'count']})
pivot_t.columns = ['bad', 'total']

# контроль полноты разбиения на категории :
unprocessed = number_data_rows - pivot_t['total'].sum()
if unprocessed == 0:
    print('Обработан весь набор данных')
else:
    print('Количество необработанных строк:', unprocessed)

print('Метрика - процент невозвращенных кредитов') # выводим метрику - процент невозврата в %
pivot_t['metric'] = pivot_t['bad'] / pivot_t['total'] * 100
pivot_t['metric'] = pivot_t['metric'].round(1) 
pivot_t.sort_values(by='metric')

Обработан весь набор данных
Метрика - процент невозвращенных кредитов


,bad,total,metric
purpose_category,,,
"недвижимость жилая, ремонт",35,609,5.7
недвижимость жилая,595,8213,7.2
недвижимость коммерческая,150,1958,7.7
свадьба,183,2337,7.8
образование вообще,280,3095,9.0
автомобиль,400,4288,9.3
образование дополнительное,89,902,9.9


### Вывод

- Самая ненадежная категория - кредиты на получение образования или на приобретение автомобиля

- Самая надежная категория - кредиты, связанные с недвижимостью. Есть незначительное отличие в случае коммерческой или некоммерческой недвижимости. Однако для достоверного вывода необходимо больше данных по кредитам, связанным с коммерческой недвижимостью. 

- Кредиты на ремонт недвижимости самые надежные, однако данных очень мало, чтобы считать такой вывод обоснованным

- Кредиты на проведение свадьбы чуть хуже кредитов, связанных с недвижимостью, но заметно лучше кредитов на автомобиль или образование 

### Шаг 4. Общий вывод

Анализ данных выявил зависимость платежеспособности клиентов от семейного положения, кол-ва детей, уровня дохода и целей получения кредита. Подробные выводы о найденных зависимостях приведены выше с соответствующими сводными таблицами. Здесь ограничимся основными результатами:

- Распределение платежеспособности в зависимости от доходов представляет из себя колоколообразную кривую с максимум показателя невозвратов у клиентов с доходами от 140 тыс. до 200 тыс. 
- Самые неплатежеспособные клиенты - это не женатые / не замужние, либо состоящие в гражданском браке. Люди, заключившие брак официально или имевшие опыт брака (разведенные / вдовствущие), заметно дисциплинированнее в возврате кредитов.
- Бездетные лучше возвращают кредиты, чем клиенты с детьми. При этом клиенты с 1-2 детьми, похоже, хуже возвращают кредиты, чем клиенты с 3 и более детьми.
- Кредиты, цели получения которых связаны с недвижимостью, самые надежные. Самые ненадежные - это кредиты на получение образования и покупку автомобиля